In [1]:
"""
Dieses Pythonscript lädt über eine SRU-Abfrage die Metadaten (MARCXML) der Titel herunter,
welche mit "bbggr" kodiert sind, extrahiert die Titeldaten und die GND-ID für den Ort,
anhand welcher nachher die Wikidata-ID (Q-ID) gesucht wird.
Mit der Q-ID können dann Daten wie Koordinaten, Webseiten etc.
aus Wikidata heruntergeladen werden.
"""

# Import der benötigten Module
import requests
import urllib.request
from lxml import etree
from bs4 import BeautifulSoup as soup
import pandas as pd
import unicodedata
import json
import numpy as np
import datetime
from google.colab import files

# SRU

# SRU-Abfrage der Metadaten (MARCXML) der Titel, welche mit "bbggr" kodiert sind
def slsp_sru(query):

    base_url = 'https://slsp-ube.alma.exlibrisgroup.com/view/sru/41SLSP_UBE'

    params = {'version': '1.2',
          'operation': 'searchRetrieve',
          'recordSchema' : 'marcxml',
          'maximumRecords': '50',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")

    records = xml.find_all('record', xmlns='http://www.loc.gov/MARC21/slim')
    num_results = len(records)
    i = 1
    while num_results == 50:
        i+=50
        params.update({'startRecord': i})
        r = requests.get(base_url, params=params)
        xml = soup(r.content, features="xml")
        new_records = xml.find_all('record', xmlns='http://www.loc.gov/MARC21/slim')
        records+=new_records
        num_results = len(new_records)

    return records

# Die eigentliche Query der SRU-Abfrage: "alma.local_field_990=bbggr"
records = slsp_sru("alma.local_field_990=bbggr")

# Für schnelles Testen kann die SRU-Abfrage z.B. auf nach 2017 publizierte Titel eingeschränkt werden.
  # Die knapp 30 Treffer sind immer noch repräsentativ, aber die Laufzeit sinkt auf 30-130 Sekunden.
#records = slsp_sru("alma.local_field_990=bbggr and alma.main_pub_date>2017")

# MARCXML

# MARCXML-parsing, also Auswertung der gewünschten MARC Kontroll- & Unterfelder aus dem XML
def parse_record_with_namespace_v2(record):

    ns = {'marc':'http://www.loc.gov/MARC21/slim'}
    xml = etree.fromstring(unicodedata.normalize('NFC', str(record)))

# MMSID: 001 (idn)
    idn = xml.xpath('marc:controlfield[@tag = "001"]', namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'ID-fail'

# Titel: 245$a (title)
    title = xml.xpath('marc:datafield[@tag = "245"]/marc:subfield[@code = "a"]', namespaces=ns)
    try:
        title = title[0].text
    except:
        title = '-'

# Verfasser: 100$a (autor)
    autor = xml.xpath('marc:datafield[@tag = "100"]/marc:subfield[@code = "a"]', namespaces=ns)
    try:
        autor = autor[0].text
    except:
        autor = '-'

# Verlag: 264$b (edit)
    edit = xml.xpath('marc:datafield[@tag = "264"]/marc:subfield[@code = "b"]', namespaces=ns)
    try:
        edit = edit[0].text
    except:
        edit = '-'

# Verlagsort: 264$a (v_ort)
    v_ort = xml.xpath('marc:datafield[@tag = "264"]/marc:subfield[@code = "a"]', namespaces=ns)
    try:
        v_ort = v_ort[0].text
    except:
        vort = '-'

# Jahr: 264$c (year)
    year = xml.xpath('marc:datafield[@tag = "264"]/marc:subfield[@code = "c"]', namespaces=ns)
    try:
        year = year[0].text
    except:
        year = '-'

# GND: 651$0 (gnd)
    gnd = xml.xpath('marc:datafield[@tag = "651"]/marc:subfield[@code = "0"]', namespaces=ns)
    if len(gnd) > 0:
        gnd = [element.text for element in gnd]
    else:
        gnd = 'no gnd'

# Ergebnissset (hier können die Spaltennamen definiert werden)
    meta_dict = {'MMSID':idn,
                 'Titel':title,
                 'Verfasser':autor,
                 'Verlag':edit,
                 'Verlagsort':v_ort,
                 'Jahr':year,
                 'GND':gnd}

    return meta_dict

# Der Output der SRU wird ins Dataframe geschrieben
sru_out = [parse_record_with_namespace_v2(record) for record in records]
df = pd.DataFrame(sru_out)

# Aufbereiten & Bereinigen der GND-IDs

# Mehrfache 651$0 in einem Titel werden durch explode in einzelne Zeilen aufgeteilt
df = df.explode('GND').explode('GND')
df = df.reset_index()

# RERO- & IDREF-IDs entfernen
df = df.replace("(\(RERO\)A.{9}|\(IDREF\).{9})",'', regex=True)

# Einträge für "Kanton Bern" (4005765-3) entfernen
df = df.replace(["\(DE-588\)4005765-3"], "", regex=True)

# GND-ID Vorspann (DE-588) entfernen
df = df.replace("\(DE-588\)",'', regex=True)

# Zeilen ohne ID entfernen
df = df[df.GND != '']

# Dataframe bereinigen
df = df.reset_index()
df = df.drop(["level_0", "index"], axis = 1)

# BEOT (Berner OrtsTabelle)

# BEOT aus GitHub ins Dataframe einlesen
github_raw_url = 'https://github.com/Berner-Ortsgeschichten/BEOT/raw/main/BEOT_20231101.xlsx'
beot = pd.read_excel(github_raw_url, sheet_name=0)

# Leere Felder mit NaN füllen
beot = beot.replace('', np.nan)

# Das bisherige Dataframe wird mit den Spalten aus der BEOT ergänzt
df = df.join(beot.set_index('GND_ID'), on='GND')

# LOD-Anreicherung aus Wikidata

baseurl = "https://www.wikidata.org/wiki/Special:EntityData/"

# Neue Spalten für die Daten aus Wikidata definieren
df['Breite'] = ''
df['Laenge'] = ''
df['CoatOfArms'] = ''
df['Webseite der Gemeinde'] = ''
df['Wikipedia'] = ''
df['geo.admin.ch'] = ''

# Anreichern mit Wikidata-Properties
for i in df.index:
    if type(df['Q_ID'][i]) == str:
        wd = df['Q_ID'][i]
        webadd = baseurl + wd + '.json'
        r = requests.get(webadd)

        try:
            data = json.loads(r.text)
        except:
            pass

        try:
            if 'P625' in data['entities'][wd]['claims']:
                df['Breite'][i] = data['entities'][wd]['claims']['P625'][0]['mainsnak']['datavalue']['value']['latitude']
                df['Laenge'][i] = data['entities'][wd]['claims']['P625'][0]['mainsnak']['datavalue']['value']['longitude']
            if 'P94' in data['entities'][wd]['claims']:
                df['CoatOfArms'][i] = data['entities'][wd]['claims']['P94'][0]['mainsnak']['datavalue']['value']
            if 'P856' in data['entities'][wd]['claims']:
                df['Webseite der Gemeinde'][i] = data['entities'][wd]['claims']['P856'][0]['mainsnak']['datavalue']['value']
            if 'dewiki' in data['entities'][wd]['sitelinks']:
                df['Wikipedia'][i] = data['entities'][wd]['sitelinks']['dewiki']['url']
            if 'P1325' in data['entities'][wd]['claims']:
                df['geo.admin.ch'][i] = data['entities'][wd]['claims']['P1325'][0]['mainsnak']['datavalue']['value']
        except KeyError:
            pass
    else:
        pass

# Leere Felder mit NaN füllen
df = df.replace('', np.nan)

# Werte zu Links anreichern
df['Wappen'] = df['CoatOfArms'].apply(lambda x: "https://commons.wikimedia.org/wiki/File:" + str(x) if pd.notnull(x) else '-')
df['HLS'] = df['id_hls'].apply(lambda x: "https://hls-dhs-dss.ch/de/articles/" + str(x) if pd.notnull(x) else '-')
df['ortsnamen.ch'] = df['id_ortsnamen'].apply(lambda x: "https://search.ortsnamen.ch/de/record/" + str(x) if pd.notnull(x) else '-')
df['cat'] = df['MMSID'].apply(lambda x: "https://slsp-ube.primo.exlibrisgroup.com/permalink/41SLSP_UBE/17e6d97/alma/" + str(x) if pd.notnull(x) else '-')
df['Wikidata'] = df['Q_ID'].apply(lambda x: "https://www.wikidata.org/wiki/" + str(x) if pd.notnull(x) else '-')

# DF ordnen
new_order = ['Ort', 'Wappen', 'Webseite der Gemeinde', 'Wikipedia', 'HLS', 'ortsnamen.ch', 'geo.admin.ch', 'Wikidata',\
             'Titel', 'Verfasser', 'Verlag', 'Verlagsort', 'Jahr', 'cat',\
             'Breite', 'Laenge', 'GND', 'Q_ID', 'id_hls', 'id_ortsnamen', 'MMSID']
df = df[new_order]

# Output & Anzeigen

# Aktuelles Datum ermitteln
current_date = datetime.datetime.now().strftime("%Y%m%d")

# Ausgabe als xlsx
output_filename = f"beog75_{current_date}.xlsx"
df.to_excel(output_filename, index=False)
files.download(output_filename)

"""
# Ausgabe als json
json_output = f"beog75_{current_date}.json"
df.to_json(json_output, orient='records', force_ascii=False)
files.download(json_output)
"""

#Ergebnisse anzeigen
print(len(df), 'Zeilen')
df


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

562 Zeilen


,Ort,Wappen,Webseite der Gemeinde,Wikipedia,HLS,ortsnamen.ch,geo.admin.ch,Wikidata,Titel,Verfasser,...,Verlagsort,Jahr,cat,Breite,Laenge,GND,Q_ID,id_hls,id_ortsnamen,MMSID
0,Sigriswil,https://commons.wikimedia.org/wiki/File:Sigris...,http://www.sigriswil.ch,https://de.wikipedia.org/wiki/Sigriswil,https://hls-dhs-dss.ch/de/articles/000537,https://search.ortsnamen.ch/de/record/309058578,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q65732,"""E Hutte voll Zyt""","Lindt, Thomas",...,Interlaken,[2017],https://slsp-ube.primo.exlibrisgroup.com/perma...,46.716389,7.716667,4054953-7,Q65732,000537,309058578,99116858901405511
1,Muri bei Bern,https://commons.wikimedia.org/wiki/File:Muri b...,http://www.muri-guemligen.ch,https://de.wikipedia.org/wiki/Muri_bei_Bern,https://hls-dhs-dss.ch/de/articles/000214,https://search.ortsnamen.ch/de/record/309006859,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q69765,"""Weisch no""","Bosshard, Fritz",...,Gümligen,2003,https://slsp-ube.primo.exlibrisgroup.com/perma...,46.931944,7.487222,4101790-0,Q69765,000214,309006859,99116915175905511
2,Schwarzenburg,https://commons.wikimedia.org/wiki/File:Schwar...,https://www.schwarzenburg.ch,https://de.wikipedia.org/wiki/Schwarzenburg_BE,https://hls-dhs-dss.ch/de/articles/008370,https://search.ortsnamen.ch/de/record/309049660,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q70708,[Schwarzenburger Ansichten],-,...,Schwarzenburg,2000,https://slsp-ube.primo.exlibrisgroup.com/perma...,46.816667,7.333333,4238357-2,Q70708,008370,309049660,99116771141905511
3,Bern-Bümpliz,https://commons.wikimedia.org/wiki/File:Bümpli...,NaN,https://de.wikipedia.org/wiki/B%C3%BCmpliz,https://hls-dhs-dss.ch/de/articles/003274,https://search.ortsnamen.ch/de/record/309005339,NaN,https://www.wikidata.org/wiki/Q1020746,100 Jahre Bern Bümpliz,"Gerber, Roland",...,Bern,2019,https://slsp-ube.primo.exlibrisgroup.com/perma...,46.943650,7.389320,4348230-2,Q1020746,003274,309005339,99116808877605511
4,Wyssachen,https://commons.wikimedia.org/wiki/File:Wyssac...,http://www.wyssachen.ch,https://de.wikipedia.org/wiki/Wyssachen,https://hls-dhs-dss.ch/de/articles/000556,https://search.ortsnamen.ch/de/record/309063253,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q66446,100 Jahre Wyssachen,-,...,[Wyssachen],[2008],https://slsp-ube.primo.exlibrisgroup.com/perma...,47.074020,7.821400,4550339-4,Q66446,000556,309063253,99116772364105511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Thun,https://commons.wikimedia.org/wiki/File:CHE Th...,http://www.thun.ch,https://de.wikipedia.org/wiki/Thun,https://hls-dhs-dss.ch/de/articles/000541,https://search.ortsnamen.ch/de/record/309059469,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q68978,Zum Beispiel Thun,"Helmle, Christian",...,Thun,2003,https://slsp-ube.primo.exlibrisgroup.com/perma...,46.759000,7.630000,4059991-7,Q68978,000541,309059469,99116877987105511
557,Gals,https://commons.wikimedia.org/wiki/File:Gals-c...,https://www.gals.ch,https://de.wikipedia.org/wiki/Gals,https://hls-dhs-dss.ch/de/articles/000283,https://search.ortsnamen.ch/de/record/309015021,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q67241,Zur Erinnerung,-,...,Gals,Juni 2016,https://slsp-ube.primo.exlibrisgroup.com/perma...,47.027100,7.048900,4134445-5,Q67241,000283,309015021,99117331893505511
558,Leissigen,https://commons.wikimedia.org/wiki/File:Leissi...,https://www.leissigen.ch,https://de.wikipedia.org/wiki/Leissigen,https://hls-dhs-dss.ch/de/articles/000339,https://search.ortsnamen.ch/de/record/309024822,https://ld.geo.admin.ch/boundaries/municipalit...,https://www.wikidata.org/wiki/Q65300,Zur Geschichte der Kirchgemeinde Leissigen,-,...,[],1996,https://slsp-ube.primo.exlibrisgroup.com/perma...,46.649722,7.766389,4520788-4,Q65300,000339,309024822,99116